In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.framework import graph_util
import numpy as np

In [2]:
IMAGE_SIZE = 28
IMAGE_CHANNEL_NUM = 1

INPUT_SIZE = IMAGE_SIZE * IMAGE_SIZE
OUTPUT_SIZE = 10

CONV_1_SIZE = 5
CONV_1_DEPTH = 32

CONV_2_SIZE = 5
CONV_2_DEPTH = 64

FC_SIZE = 512

In [3]:
def get_weight(shape):
    return tf.get_variable("weight", shape, initializer=tf.truncated_normal_initializer(stddev=0.1))

In [4]:
def get_bias(shape):
    return tf.get_variable("bias", shape, initializer=tf.constant_initializer(0.0))

In [5]:
def conv2d(input_tensor, weight):
    return tf.nn.conv2d(input_tensor, weight, strides=[1, 1, 1, 1], padding="SAME")

In [6]:
def max_pool_2x2(input_tensor):
    return tf.nn.max_pool(input_tensor, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

In [7]:
def inference(input_tensor, regularizer=None):
    with tf.variable_scope("layer_1_conv"):
        conv_1_weight = get_weight([CONV_1_SIZE, CONV_1_SIZE, IMAGE_CHANNEL_NUM, CONV_1_DEPTH])
        conv_1_bias = get_bias([CONV_1_DEPTH])
        conv_1 = conv2d(input_tensor, conv_1_weight)
        conv_1_activation = tf.nn.relu(tf.nn.bias_add(conv_1, conv_1_bias))
    with tf.variable_scope("layer_1_pool"):
        pool_1 = max_pool_2x2(conv_1_activation)
    with tf.variable_scope("layer_2_conv"):
        conv_2_weight = get_weight([CONV_2_SIZE, CONV_2_SIZE, CONV_1_DEPTH, CONV_2_DEPTH])
        conv_2_bias = get_bias([CONV_2_DEPTH])
        conv_2 = conv2d(pool_1, conv_2_weight)
        conv_2_activation = tf.nn.relu(tf.nn.bias_add(conv_2, conv_2_bias))
    with tf.variable_scope("layer_2_pool"):
        pool_2 = max_pool_2x2(conv_2_activation)
        pool_2_shape = pool_2.get_shape().as_list()
        nodes = pool_2_shape[1] * pool_2_shape[2] * pool_2_shape[3]
        pool_2_reshaped = tf.reshape(pool_2, [-1, nodes])
    with tf.variable_scope("layer_3_fc"):
        fc_1_weight = get_weight([nodes, FC_SIZE])
        if regularizer != None:
            tf.add_to_collection("losses", regularizer(fc_1_weight))
        fc_1_bias = get_bias([FC_SIZE])
        fc_1 = tf.matmul(pool_2_reshaped, fc_1_weight) + fc_1_bias
        fc_1_activation = tf.nn.relu(fc_1)
    with tf.variable_scope("layer_4_fc"):
        fc_2_weight = get_weight([FC_SIZE, OUTPUT_SIZE])
        if regularizer != None:
            tf.add_to_collection("losses", regularizer(fc_2_weight))
        fc_2_bias = get_bias([OUTPUT_SIZE])
        logits = tf.matmul(fc_1_activation, fc_2_weight) + fc_2_bias
    return logits

In [8]:
BATCH_SIZE = 100
LEARNING_RATE = 0.01
REGULARIZATION_RATE = 0.0001
# TRAINING_STEPS = 3000
TRAINING_STEPS = 200
MOVING_AVERAGE_DECAY = 0.99

In [9]:
x = tf.placeholder(tf.float32, [None, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNEL_NUM], name="x")
y_ = tf.placeholder(tf.float32, [None, OUTPUT_SIZE], name="y_")
regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
y = inference(x, regularizer)
global_step = tf.Variable(0, trainable=False)
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
loss = tf.reduce_mean(cross_entropy) + tf.add_n(tf.get_collection("losses"))
train_op = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss, global_step=global_step)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

output = tf.identity(tf.nn.softmax(y, axis=1), name="output")
print(x, output)

Tensor("x:0", shape=(?, 28, 28, 1), dtype=float32) Tensor("output:0", shape=(?, 10), dtype=float32)


In [10]:
mnist = input_data.read_data_sets("./data", one_hot=True)

Extracting ./data/train-images-idx3-ubyte.gz
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz


In [ ]:
def reshape_image(images, num):
    return np.reshape(images, [num, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNEL_NUM])

In [ ]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(1, TRAINING_STEPS + 1):
        xs, ys = mnist.train.next_batch(BATCH_SIZE)
        xs_reshaped = reshape_image(xs, BATCH_SIZE)
        x_validate = reshape_image(mnist.validation.images, mnist.validation.num_examples)
        _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: xs_reshaped, y_: ys})
        if i % 50 == 0:
            accuracy_value = sess.run(accuracy, feed_dict={x: x_validate, y_: mnist.validation.labels})
            print("Setp %d, loss = %g, accuracy = %g." % (step, loss_value, accuracy_value))
    graph_def = tf.get_default_graph().as_graph_def()
    output_graph = graph_util.convert_variables_to_constants(sess, graph_def, ["output"])
    with tf.gfile.GFile("./mnist.pb", "wb") as f:
        f.write(output_graph.SerializeToString())

Setp 50, loss = 1.17767, accuracy = 0.7772.
Setp 100, loss = 1.03495, accuracy = 0.8768.


```
TENSORFLOW_REPO_DIR/bazel-bin/tensorflow/contrib/lite/toco/toco  \
  --input_file=mnist.pb \
  --input_format=TENSORFLOW_GRAPHDEF  --output_format=TFLITE \
  --output_file=mnist.tflite --inference_type=FLOAT \
  --input_type=FLOAT --input_arrays=x \
  --output_arrays=output --input_shapes=1,28,28,1
```